In [0]:
dbutils.fs.mount(
       source = "wasbs://firedataraw@firesncidentstorage.blob.core.windows.net",
       mount_point = "/mnt/data",
       extra_configs = {"fs.azure.account.key.firesncidentstorage.blob.core.windows.net":"SrKWR/sqlz7NXM3mPnQxCQK0yIO+vEtDrniW1OQAhdKuEAbCBCmTaei/ne7duDflKIivlkabdaL9+AStSdciug=="}
)

In [0]:
%fs
ls "/mnt/data"

path,name,size,modificationTime
dbfs:/mnt/data/processeddata/,processeddata/,0,0
dbfs:/mnt/data/sink/,sink/,0,0


In [0]:
%fs
ls "/mnt/data/sink"

path,name,size,modificationTime
dbfs:/mnt/data/sink/fire_incident.csv,fire_incident.csv,328582222,1708779341000


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [0]:
df = spark.read.csv("/mnt/data/sink/fire_incident.csv", inferSchema=True, header=True)

In [0]:
df.show(5)

+--------------+----------+-------+-------------------+----------+---------------+-------------------+--------------------+----------------+--------------------+--------------------+-------------+-----------------+---------------+--------+------------------+------------------+----------+---------------+--------------------+--------------------+---------+----------+---------------+----------------+-----------+------------+------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+------------------+-----------------+--------+
|IncidentNumber|DateOfCall|CalYear|         TimeOfCall|HourOfCall|  IncidentGroup|StopCodeDescription|  SpecialServiceType|PropertyCategory|        PropertyType|    AddressQualifier|Postcode_full|Postcode_district|           UPRN|    USRN|IncGeo_BoroughCode|IncGeo_BoroughName|ProperCase|IncGeo_Wa

In [0]:
df.printSchema()

root
 |-- IncidentNumber: string (nullable = true)
 |-- DateOfCall: string (nullable = true)
 |-- CalYear: integer (nullable = true)
 |-- TimeOfCall: timestamp (nullable = true)
 |-- HourOfCall: integer (nullable = true)
 |-- IncidentGroup: string (nullable = true)
 |-- StopCodeDescription: string (nullable = true)
 |-- SpecialServiceType: string (nullable = true)
 |-- PropertyCategory: string (nullable = true)
 |-- PropertyType: string (nullable = true)
 |-- AddressQualifier: string (nullable = true)
 |-- Postcode_full: string (nullable = true)
 |-- Postcode_district: string (nullable = true)
 |-- UPRN: string (nullable = true)
 |-- USRN: string (nullable = true)
 |-- IncGeo_BoroughCode: string (nullable = true)
 |-- IncGeo_BoroughName: string (nullable = true)
 |-- ProperCase: string (nullable = true)
 |-- IncGeo_WardCode: string (nullable = true)
 |-- IncGeo_WardName: string (nullable = true)
 |-- IncGeo_WardNameNew: string (nullable = true)
 |-- Easting_m: string (nullable = true)


In [0]:
df = df.select(["IncidentNumber", "CalYear", "DateOfCall", "HourOfCall", "IncidentGroup", "StopCodeDescription", "IncGeo_BoroughCode",
                "IncGeo_BoroughName", "IncGeo_WardCode", "IncGeo_WardName", 
                "SpecialServiceType", "PropertyCategory", "PropertyType", "Postcode_district",
                 "IncidentStationGround", "FirstPumpArriving_AttendanceTime", "FirstPumpArriving_DeployedFromStation",
                "SecondPumpArriving_AttendanceTime", "SecondPumpArriving_DeployedFromStation", "NumStationsWithPumpsAttending",
                "NumPumpsAttending", "PumpCount", "PumpMinutesRounded", "Notional Cost (£)", "NumCalls"])

In [0]:
df.show(5)

+--------------+-------+----------+----------+---------------+-------------------+------------------+------------------+---------------+--------------------+--------------------+----------------+--------------------+-----------------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+------------------+-----------------+--------+
|IncidentNumber|CalYear|DateOfCall|HourOfCall|  IncidentGroup|StopCodeDescription|IncGeo_BoroughCode|IncGeo_BoroughName|IncGeo_WardCode|     IncGeo_WardName|  SpecialServiceType|PropertyCategory|        PropertyType|Postcode_district|IncidentStationGround|FirstPumpArriving_AttendanceTime|FirstPumpArriving_DeployedFromStation|SecondPumpArriving_AttendanceTime|SecondPumpArriving_DeployedFromStation|NumStationsWithPumpsAttending|NumPumpsAttending|PumpCount|PumpMinutesRounded|Notional Cost (£)|

In [0]:
df = df.withColumn("DateOfCall", F.to_date(F.col("DateOfCall"), "dd-MMM-yy")) \
     .withColumn("FirstPumpArriving_AttendanceTime", F.col("FirstPumpArriving_AttendanceTime").cast(T.IntegerType())) \
     .withColumn("SecondPumpArriving_AttendanceTime", F.col("SecondPumpArriving_AttendanceTime").cast(T.IntegerType())) \
     .withColumn("NumStationsWithPumpsAttending", F.col("NumStationsWithPumpsAttending").cast(T.IntegerType())) \
     .withColumn("NumPumpsAttending", F.col("NumPumpsAttending").cast(T.IntegerType())) \
     .withColumn("NumCalls", F.col("NumCalls").cast(T.IntegerType()))

In [0]:
col_timestamp = []
dtyp_col = []
for c, dtyp in df.dtypes:
    if dtyp == "date":
        col_timestamp.append(c)
    else:
        dtyp_col.append(c)

In [0]:
df.select([F.count(F.when( F.col(c).isNull() | F.isnan(c) , c)).alias(c)  
           for c in dtyp_col]).show()

+--------------+-------+----------+-------------+-------------------+------------------+------------------+---------------+---------------+------------------+----------------+------------+-----------------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+------------------+-----------------+--------+
|IncidentNumber|CalYear|HourOfCall|IncidentGroup|StopCodeDescription|IncGeo_BoroughCode|IncGeo_BoroughName|IncGeo_WardCode|IncGeo_WardName|SpecialServiceType|PropertyCategory|PropertyType|Postcode_district|IncidentStationGround|FirstPumpArriving_AttendanceTime|FirstPumpArriving_DeployedFromStation|SecondPumpArriving_AttendanceTime|SecondPumpArriving_DeployedFromStation|NumStationsWithPumpsAttending|NumPumpsAttending|PumpCount|PumpMinutesRounded|Notional Cost (£)|NumCalls|
+--------------+-------+----------+-----------

In [0]:
df.select([F.count(F.when( F.col(c).isNull() , c)).alias(c)  
           for c in col_timestamp]).show()

+----------+
|DateOfCall|
+----------+
|         0|
+----------+



In [0]:
from pyspark.sql import Window as w

district_partition = w.partitionBy(F.col("Postcode_district"))

df_mod = df.withColumn("SpecialServiceType", F.when(F.col("SpecialServiceType").isNull(), "Not a special service").otherwise(F.col("SpecialServiceType"))) \
            .withColumn("FirstPumpArriving_AttendanceTime",F.when(F.col("FirstPumpArriving_AttendanceTime").isNull() | F.isnan("FirstPumpArriving_AttendanceTime"), F.avg(F.col("FirstPumpArriving_AttendanceTime")).over(district_partition)).otherwise(F.col("FirstPumpArriving_AttendanceTime"))) \
            .withColumn("FirstPumpArriving_DeployedFromStation", F.when(F.col("FirstPumpArriving_DeployedFromStation").isNull(), "Not Known").otherwise(F.col("FirstPumpArriving_DeployedFromStation"))) \
            .withColumn("SecondPumpArriving_AttendanceTime", F.when(F.col("SecondPumpArriving_AttendanceTime").isNull() | F.isnan("SecondPumpArriving_AttendanceTime"), F.avg(F.col("SecondPumpArriving_AttendanceTime")).over(district_partition)).otherwise(F.col("SecondPumpArriving_AttendanceTime"))) \
            .withColumn("SecondPumpArriving_DeployedFromStation", F.when(F.col("SecondPumpArriving_DeployedFromStation").isNull(), "Not Known").otherwise(F.col("SecondPumpArriving_DeployedFromStation"))) \
            .withColumn("NumStationsWithPumpsAttending", F.when(F.col("NumStationsWithPumpsAttending").isNull() | F.isnan("NumStationsWithPumpsAttending"), F.avg(F.col("NumStationsWithPumpsAttending")).over(district_partition)).otherwise(F.col("NumStationsWithPumpsAttending"))) \
            .withColumn("NumPumpsAttending", F.when(F.col("NumPumpsAttending").isNull() | F.isnan("NumPumpsAttending"), F.avg(F.col("NumPumpsAttending")).over(district_partition)).otherwise(F.col("NumPumpsAttending"))) \
            .withColumn("NumCalls", F.when(F.col("NumCalls").isNull() | F.isnan("NumCalls"), F.avg(F.col("NumCalls")).over(district_partition)).otherwise(F.col("NumCalls"))) 

In [0]:
df_mod.columns

['IncidentNumber',
 'CalYear',
 'DateOfCall',
 'HourOfCall',
 'IncidentGroup',
 'StopCodeDescription',
 'IncGeo_BoroughCode',
 'IncGeo_BoroughName',
 'IncGeo_WardCode',
 'IncGeo_WardName',
 'SpecialServiceType',
 'PropertyCategory',
 'PropertyType',
 'Postcode_district',
 'IncidentStationGround',
 'FirstPumpArriving_AttendanceTime',
 'FirstPumpArriving_DeployedFromStation',
 'SecondPumpArriving_AttendanceTime',
 'SecondPumpArriving_DeployedFromStation',
 'NumStationsWithPumpsAttending',
 'NumPumpsAttending',
 'PumpCount',
 'PumpMinutesRounded',
 'Notional Cost (£)',
 'NumCalls']

In [0]:
fact_table_col = ["IncidentNumber", "CalYear", "DateOfCall", "HourOfCall", "IncidentGroup",
              "StopCodeDescription", "SpecialServiceType", "PropertyType",
              "PropertyCategory","Postcode_district", "IncGeo_WardName", "IncGeo_BoroughName","IncGeo_BoroughCode", "IncGeo_WardCode", "IncidentStationGround",
              "FirstPumpArriving_AttendanceTime", "FirstPumpArriving_DeployedFromStation",
              "SecondPumpArriving_AttendanceTime", "SecondPumpArriving_DeployedFromStation",
              "NumStationsWithPumpsAttending", "NumPumpsAttending", "PumpCount",
              "PumpMinutesRounded", "Notional Cost (£)", "NumCalls"]

In [0]:
fact_table_df = df_mod.select(fact_table_col)

In [0]:
fact_table_df.repartition(1).write.option("header","true").option("mode", "overwrite").csv("/mnt/data/processeddata/fire_incident")

In [0]:
%fs
ls /mnt/data/processeddata/fire_incident

path,name,size,modificationTime
dbfs:/mnt/data/processeddata/fire_incident/_SUCCESS,_SUCCESS,0,1708860486000
dbfs:/mnt/data/processeddata/fire_incident/_committed_193762442563937514,_committed_193762442563937514,111,1708860485000
dbfs:/mnt/data/processeddata/fire_incident/_started_193762442563937514,_started_193762442563937514,0,1708860477000
dbfs:/mnt/data/processeddata/fire_incident/part-00000-tid-193762442563937514-27eb4d9c-dea6-48ed-95f0-63658720d412-35-1-c000.csv,part-00000-tid-193762442563937514-27eb4d9c-dea6-48ed-95f0-63658720d412-35-1-c000.csv,224519281,1708860485000


In [0]:
dbutils.fs.unmount("/mnt/data")

/mnt/data has been unmounted.


True